In [1]:
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde
import seaborn as sns
import scipy.io
import h5py
import mat73
from tqdm import tqdm 
import matplotlib.pyplot as plt

from pynwb import NWBHDF5IO

import pyarrow as pa
import pyarrow.parquet as pq

import torch

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
animal_id = 'Jenkins'
session_id = '20090916'

file = h5py.File(f'/home/makki/data/NWB/000070/sub-Jenkins/sub-{animal_id}_ses-{session_id}_behavior+ecephys.nwb')
io = NWBHDF5IO(file=file, mode='r', load_namespaces=True)

/home/makki/anaconda3/envs/opium/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.3.0 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/makki/anaconda3/envs/opium/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.2.5 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


In [3]:
nwb = io.read()
nwb

,x,y,z,imp,location,filtering,group,group_name,gain,offset
id,,,,,,,,,,
1,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x140303135109904\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x140303134738640\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
2,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x140303135109904\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x140303134738640\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
3,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x140303135109904\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x140303134738640\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
4,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x140303135109904\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x140303134738640\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
,x,y,z,imp,location,filtering,group,group_name,gain,offset
id,,,,,,,,,,
1,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x140303135109904\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x140303134738640\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
2,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x140303135109904\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x140303134738640\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0
3,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x140303135109904\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x140303134738640\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,1.0,0.0


In [4]:
trialtable = nwb.trials.to_dataframe()
trialtable

,start_time,stop_time,target_presentation_time,go_cue_time,reaction_time,move_begins_time,move_ends_time,discard_trial,task_success,trial_type,...,correct_reach,maze_num_targets,maze_num_barriers,novel_maze,target_positions,frame_details,hit_target_position,target_size,barrier_info,timeseries
id,,,,,,,,,,,,,,,,,,,,,
0,8.853967,12.693967,9.751967,10.498967,0.375,10.830967,11.295967,0,1,6,...,1,3,9,0,"[[133, -81], [-118, -83], [-77, 82]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[133, -81]",13,"[[-33.0, 47.0, 6.0, 37.0], [-77.0, 48.0, 11.0,...","[(0, 3141, Eye pynwb.behavior.SpatialSeries at..."
1,17.774200,21.348967,18.490200,19.037200,0.340,19.346200,19.881200,0,1,2,...,1,3,8,0,"[[132, 56], [131, -55], [-118, -7]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[-118, -7]",13,"[[-64.0, -47.0, 113.0, 7.0], [-64.0, -39.0, 40...","[(3141, 2806, Eye pynwb.behavior.SpatialSeries..."
2,22.143867,25.718733,22.941867,23.905867,0.335,24.200867,24.500867,0,1,2,...,1,1,0,0,"[[-118, -7]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[-118, -7]",13,[],"[(5947, 3056, Eye pynwb.behavior.SpatialSeries..."
3,26.579000,29.914000,27.361000,27.858000,0.350,28.176000,28.591000,0,1,3,...,1,3,8,0,"[[132, 56], [131, -55], [-118, -7]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[131, -55]",13,"[[-64.0, -47.0, 113.0, 7.0], [-64.0, -39.0, 40...","[(9003, 2711, Eye pynwb.behavior.SpatialSeries..."
4,30.578867,34.048767,31.528867,32.226867,0.310,32.495867,32.825867,0,1,6,...,1,1,0,0,"[[133, -81]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[133, -81]",13,[],"[(11714, 2946, Eye pynwb.behavior.SpatialSerie..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260,12368.759931,12373.144565,12369.700931,12370.197931,0.560,12371.431931,12371.801931,0,1,5,...,0,3,9,0,"[[133, -81], [-118, -83], [-77, 82]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[-77, 82]",13,"[[-33.0, 47.0, 6.0, 37.0], [-77.0, 48.0, 11.0,...","[(6806955, 3741, Eye pynwb.behavior.SpatialSer..."
2261,12373.914598,12376.944465,12374.819598,12375.583598,0.470,12376.016598,12376.266598,0,0,4,...,0,1,0,0,"[[132, 56]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[132, 56]",13,[],"[(6810696, 3030, Eye pynwb.behavior.SpatialSer..."
2262,12404.794665,12408.349531,12405.517665,12406.365665,0.365,12406.696665,12407.056665,0,1,12,...,1,1,0,0,"[[-130, -13]]","[[[-200]], [[200]], [[-155]], [[155]], [[20]]]","[-130, -13]",13,[],"[(6813857, 2961, Eye pynwb.behavior.SpatialSer..."


In [5]:
n_trials = len(trialtable)
print(n_trials)
n_neurons = nwb.units.to_dataframe()['spike_times'].shape[0]
print(n_neurons)

2265
192


In [6]:
trialtable.correct_reach.value_counts()

correct_reach
1    1379
0     886
Name: count, dtype: int64

In [7]:
trialtable[trialtable.maze_condition != 0].correct_reach.value_counts()

correct_reach
1    1379
0      36
Name: count, dtype: int64

In [8]:
for column in trialtable.columns:
    print(column)

start_time
stop_time
target_presentation_time
go_cue_time
reaction_time
move_begins_time
move_ends_time
discard_trial
task_success
trial_type
trial_version
proto_trial
maze_condition
correct_reach
maze_num_targets
maze_num_barriers
novel_maze
target_positions
frame_details
hit_target_position
target_size
barrier_info
timeseries


In [ ]:
neurons = nwb.units.to_dataframe()['spike_times'] # Neuron id x Spike Time
hand = nwb.processing['behavior'].data_interfaces['Position'].spatial_series['Hand'].data[:] # x, y 
cursor = nwb.processing['behavior'].data_interfaces['Position'].spatial_series['Cursor'].data[:] # x, y
session_time = np.round(nwb.processing['behavior'].data_interfaces['Position'].spatial_series['Hand'].timestamps[:] + 0.0001, decimals=3)

In [13]:
trialtable['trial_id'] = trialtable.index

trialtable = trialtable[trialtable.task_success == 1]
typ_trials = trialtable[trialtable.correct_reach == 1]
dev_trials = trialtable[trialtable.correct_reach == 0]

In [14]:
def get_avg_reach(trials, maze_cond):
    pass

In [16]:
trialtable.correct_reach.value_counts(0)

correct_reach
1    1379
0     862
Name: count, dtype: int64

In [18]:
dev_trials.trial_id.values

array([   5,    6,    7,   10,   11,   13,   17,   19,   22,   23,   26,
         28,   33,   35,   36,   39,   46,   47,   48,   51,   52,   53,
         55,   56,   61,   62,   63,   64,   65,   66,   70,   72,   80,
         82,   85,   88,   91,   92,   94,  102,  107,  110,  111,  112,
        115,  117,  119,  124,  127,  131,  132,  133,  135,  136,  137,
        138,  139,  143,  145,  146,  149,  152,  157,  158,  159,  161,
        165,  167,  169,  177,  178,  186,  187,  189,  194,  195,  199,
        202,  205,  212,  214,  217,  220,  221,  224,  225,  227,  230,
        236,  237,  239,  242,  243,  244,  245,  246,  248,  251,  252,
        256,  259,  263,  264,  265,  267,  269,  274,  276,  277,  279,
        280,  281,  284,  291,  292,  295,  305,  308,  310,  314,  316,
        317,  318,  319,  320,  321,  323,  325,  326,  329,  331,  332,
        339,  340,  345,  346,  369,  370,  373,  374,  378,  379,  381,
        385,  386,  387,  388,  392,  395,  396,  3

### Out-of-sample

In [ ]:
conds_ids, cond_counts = torch.unique(conds, return_counts=True)
n_conds = len(conds_ids)
cond_counts

In [ ]:
# Trial-averaged reaches of each condition
cxbxb = torch.zeros((n_conds, , labels.shape[2]))
# The position of the active target that the animal is supposed to reach.
cxp = torch.zeros((n_conds, 2))
# Measure how disperse trials in each condition
c_std = torch.zeros((n_conds))

for ci in range(n_conds):
    mask = conds == ci
    
    cxbxv[ci, :, :] = labels[mask, :, :].mean(axis=0)
    c_std[ci] = cxbxv[ci, :, :].std(axis=0).mean(axis=0)
    # All trials in a condition have the same specified final active target
    cxp[ci, :] = torch.tensor(true_targets[mask][0])

print(cxbxv.shape)
print(c_std.shape)
print(cxp.shape)

In [22]:
trialtable.maze_condition

id
0       12
1        3
2        1
3        6
4       10
        ..
2259    16
2260     0
2262    25
2263    20
2264     5
Name: maze_condition, Length: 2241, dtype: uint8